In [3]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType
import os
from getpass import getpass
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
import re
import ast
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

c:\Users\asimj\.conda\envs\langchain\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.12) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [4]:
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.contracts import ContractCode, ContractInstance
from substrateinterface.exceptions import SubstrateRequestException

substrate_relay = SubstrateInterface(url="wss://shibuya-rpc.dwellir.com")
substrate_contract = SubstrateInterface(
    url='wss://rococo-contracts-rpc.polkadot.io')

In [5]:
def format_balance(amount: int):
    amount = format(
        amount / 10**substrate_relay.properties.get('tokenDecimals', 0),
        ".15g")
    return f"{amount} {substrate_relay.properties.get('tokenSymbol', 'UNIT')}"

In [6]:
def get_account_balance(account_address):
    result = substrate_relay.query("System", "Account", [account_address])
    balance = (result.value["data"]["free"] + result.value["data"]["reserved"])

    return format_balance(balance)

In [7]:
def get_polkadot_account_balance(account_address):
    """Method to get the balance of a Polkadot account."""
    result = substrate_relay.query("System", "Account", [account_address])
    balance = (result.value["data"]["free"] + result.value["data"]["reserved"])

    return format_balance(balance)

In [31]:
def send_balance(recipient_address, amount):
    call = substrate_relay.compose_call(call_module='Balances',
                                        call_function='transfer',
                                        call_params={
                                            'dest': recipient_address,
                                            'value': amount * 10**15
                                        })

    extrinsic = substrate_relay.create_signed_extrinsic(
        call=call,
        keypair=Keypair.create_from_mnemonic(menemonic),
        era={'period': 64})

    try:
        receipt = substrate_relay.submit_extrinsic(extrinsic,
                                                   wait_for_inclusion=True)

        print('Extrinsic "{}" included in block "{}"'.format(
            receipt.extrinsic_hash, receipt.block_hash))

        print(receipt)

        if receipt.is_success:
            print('✅ Success, triggered events:')
            for event in receipt.triggered_events:
                print(f'* {event.value}')
        else:
            print('⚠️ Extrinsic Failed: ', receipt.error_message)

        return receipt

    except Substrate_relayRequestException as e:
        print(e)
        return False

In [67]:
def get_transfer_details(extrinsic_index):
    url = base_url + "/api/scan/extrinsic"
    headers = {"Content-Type": "application/json", "X-API-Key": api_key}
    payload = {"extrinsic_index": extrinsic_index}

    response = requests.post(url, headers=headers, json=payload)
    return response.json()

In [68]:
get_transfer_details("0x8d32669ece6e69c8bdf6b1381b566264ef38e5f0226d67eca1703238b9c88d25")

{'code': 0, 'message': 'Success', 'generated_at': 1690027071, 'data': None}

In [8]:
print(get_account_balance("5DvyRvNq5jpvjat2qkGhiKjJQdz5cwreJW5yxvLBLRpHnoGo"))

9.74665958012062 SBY


In [79]:
class GetAccountBalanceInput(BaseModel):
    """Inputs for get_account_balance"""

    account_address: str = Field(description="the address of the account to get the balance of")

class  GetAccountBalanceTool(BaseTool):
    name = "get_account_balance"
    description = """
        Useful when you want to get the balance of an polkadot account.
        The account address is the address of the account you want to get the balance of.
        The address format is ss58 encoded.
        """
    args_schema: Type[BaseModel] = GetAccountBalanceInput

    def _run(self, account_address: str):
        account_balance = get_account_balance(account_address)
        return account_balance

    def _arun(self, account_address: str):
        raise NotImplementedError("get_current_stock_price does not support async")
        
class GenerateInkPolkadotContractInput(BaseModel):
    """Inputs for generate_ink_polkadot_contract"""

    contract_description: str = Field(description="A description in simple english of what you would like the contract to do")

class GenerateInkPolkadotContractTool(BaseTool):
    name = "generate_ink_polkadot_contract"
    description = """
        Useful when you want to generate a polkadot contract in ink or just an ink contract.
        The contract description is a description of what you would like the contract to do.
        
        This also deploys the code to Shibuya Testnet.

        returns the contract address
        """
    args_schema: Type[BaseModel] = GenerateInkPolkadotContractInput

    def _run(self, contract_description: str):
        address = genCompileDeployContract(contract_description)
        return address

        
    def _arun(self, account_address: str):
        raise NotImplementedError("get_current_stock_price does not support async")

class SendSubstrateBalanceInput(BaseModel):
    """Inputs for send_substrate_balance"""

    recipient_address: str = Field(description="the address of the account to send the balance to")
    amount: float = Field(description="the amount to send.")

class SendSubstrateBalanceTool(BaseTool):
    name = "send_substrate_balance"
    description = """
        Useful when you want to send a balance to a polkadot account.
        If balance is not specified send 0.001
        We will be sending Shibuya Testnet tokens/SBY.
        returns the extrinsic hash if successful
        """
    args_schema: Type[BaseModel] = SendSubstrateBalanceInput

    def _run(self, recipient_address: str, amount: int):
        res = send_balance(recipient_address, amount)
        return res.extrinsic_hash

    def _arun(self, account_address: str):
        raise NotImplementedError("get_current_stock_price does not support async")

class ListAllTransactionsInput(BaseModel):
    """Inputs for list_all_transactions"""

    account_address: str = Field(description="the address of the account to get the transactions of")

class ListAllTransactionsTool(BaseTool):
    name = "list_all_transactions"
    description = """
        Useful when you want to list all transactions of a polkadot account.
        Lists the last first 3 and last 3 transactions.
        """
    args_schema: Type[BaseModel] = ListAllTransactionsInput

    def _run(self, account_address: str):
        res = get_account_transfers(account_address)
        return res
    
    def _arun(self, account_address: str):
        raise NotImplementedError("list_all_transactions does not support async")

class GetTransferDetailsInput(BaseModel):
    """Inputs for get_transfer_details"""

    extrinsic_hash: str = Field(description="the extrinsic hash of the transaction to get the details of, starts with 0x")

class GetTransferDetailsTool(BaseTool):
    name = "get_transfer_details"
    description = """
        Useful when you want to get the details of a transaction.
        returns code, if successful, block time and data if it exists.
        """
    args_schema: Type[BaseModel] = GetTransferDetailsInput

    def _run(self, extrinsic_hash: str):
        res = get_transfer_details(extrinsic_hash)
        return ["successfully retreived data. Data:", res]
    
    def _arun(self, account_address: str):
        raise NotImplementedError("get_transfer_details does not support async")

llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0)
tools = [GetAccountBalanceTool(), GenerateInkPolkadotContractTool(), SendSubstrateBalanceTool(), ListAllTransactionsTool(), GetTransferDetailsTool()]
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)


In [10]:
embeddings = OpenAIEmbeddings()
db = DeepLake(
    dataset_path=f"hub://commanderastern/polka-code-3",
    read_only=True,
    embedding_function=embeddings,
)

Deep Lake Dataset in hub://commanderastern/polka-code-3 already exists, loading from the storage


In [11]:
retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 20
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 20

In [12]:
def filter(x):
    # filter based on source code
    if "something" in x["text"].data()["value"]:
        return False

    # filter based on path e.g. extension
    metadata = x["metadata"].data()["value"]
    return "only_this" in metadata["source"] or "also_that" in metadata["source"]

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name="gpt-3.5-turbo-16k")  # 'ada' 'gpt-3.5-turbo' 'gpt-4',
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [15]:
import subprocess

def run_command(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    output, error = process.communicate()

    if process.returncode != 0:
        print(f"Error occurred: {error.decode().strip()}")
    else:
        print(f"Output: {output.decode().strip()}")


In [16]:
menemonic = "edge lawn cost piano narrow annual movie duty forum patient check output"

In [17]:
#setup for generating and deploy
embeddings = OpenAIEmbeddings()
db = DeepLake(
    dataset_path=f"hub://commanderastern/polka-code-2",
    read_only=True,
    embedding_function=embeddings,
)

retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 20
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 20

def filter(x):
    # filter based on source code
    if "something" in x["text"].data()["value"]:
        return False

    # filter based on path e.g. extension
    metadata = x["metadata"].data()["value"]
    return "only_this" in metadata["source"] or "also_that" in metadata["source"]
model_genContract = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temprature = 0)  # 'ada' 'gpt-3.5-turbo' 'gpt-4',

qa = ConversationalRetrievalChain.from_llm(model_genContract, retriever=retriever)
menemonic = "edge lawn cost piano narrow annual movie duty forum patient check output"

def run_command(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    output, error = process.communicate()

    if process.returncode != 0:
        print(f"Error occurred: {error.decode().strip()}")
    else:
        print(f"Output: {output.decode().strip()}")

Deep Lake Dataset in hub://commanderastern/polka-code-2 already exists, loading from the storage


WARNING! temprature is not default parameter.
                    temprature was transferred to model_kwargs.
                    Please confirm that temprature is what you intended.


In [32]:
def genCompileDeployContract(description: str):
    '''
    Generate, compile and deploy a contract to Shibuya Testnet
    '''
    questions = [
    f"""
    Give me a basic ink contract code, 
    Description of code = {description}
    """+"""
    -- There should be no print statments in the contract return everything.
    -- enclose the contract content in #startContract# and #endContract#
    -- follow this basic format when generating the code

    #startContract#
    #[ink::contract]
    mod contract_name \{

    }
    #endContract#
    """
    ]

    chat_history = []

    # extrating the contract code from the result
    result = qa({"question": questions[0], "chat_history": chat_history})
    chat_history.append((questions[0], result["answer"]))

    pattern = r'#startContract#(.*?)#endContract#'
    pattern2 = r'```rust(.*?)```'
    contract_code = re.search(pattern, result["answer"], re.DOTALL)
    res = ""
    
    if contract_code:
        res = contract_code.group(1).strip()
    else:
        contract_code = re.search(pattern2, result["answer"], re.DOTALL)
        if contract_code:
            res = contract_code.group(1).strip()
    res = r"""#![cfg_attr(not(feature = "std"), no_std, no_main)]""" + '\n'+ res
    post_process_code = re.sub(r'^\s*use.*\n?', '', res, flags=re.MULTILINE)

    post_process_code = re.sub(r'^\s*struct', 'pub struct', post_process_code, flags=re.MULTILINE)

    post_process_code = re.sub(r'^\s*#\s*\[derive\(.*\n?', '', post_process_code, flags=re.MULTILINE)

    print(post_process_code)

    # generating the constructor args
    new_function_pattern = r'(pub fn new.*?\))'
    new_function_match = re.search(new_function_pattern, post_process_code, re.DOTALL)

    if new_function_match:
        res = new_function_match.group(0)
    print(res)
    chat = ChatOpenAI()
    messages = [
        SystemMessage(content=r"""
        give the argumentsvalues for "pub fn new(value1: i32, value2: i32)" in the form of a dictionary
        -- Just the dictionary
        -- No need fore explanation or additional code
        -- empty dictionary is also fine
        -- for invalid input empty dictionary will be returned
        example: 
        Input: pub fn new(coolVal: i32)
        Output: {"coolVal": 1}"""),
        HumanMessage(content=f"{res}")
    ]
    constructor_args = ast.literal_eval(chat(messages).content)

    with open('code/lib.rs', 'w') as file:
        file.write(post_process_code)

    # compiling contract
    print(run_command("cd code && cargo contract build"))

    # Upload WASM code
    code = ContractCode.create_from_contract_files(
        metadata_file=os.path.join(os.getcwd(), 'code/target/ink',
                                'my_contract.json'),
        wasm_file=os.path.join(os.getcwd(), 'code/target/ink',
                            'my_contract.wasm'),
        substrate=substrate_relay)

    # Deploy contract
    print('Deploy contract...')
    contract = code.deploy(keypair=Keypair.create_from_mnemonic(
        menemonic),
                        constructor="new",
                        args=constructor_args,
                        value=0,
                        gas_limit={'ref_time': 25990000000, 'proof_size': 1199000},
                        upload_code=True)

    return contract.contract_address

In [24]:
agent.run(
    "I would like to build an ink contract and deploy it to Shibuya Testnet. It should be a contract that stores value 1 and 2 and return the sum of the two values"
)



> Entering new AgentExecutor chain...

Invoking: `generate_ink_polkadot_contract` with `{'contract_description': 'A contract that stores value 1 and 2 and returns the sum of the two values'}`


#![cfg_attr(not(feature = "std"), no_std, no_main)]
#[ink::contract]
mod contract_name {
    #[ink(storage)]
    pub struct Contract {
        value1: i32,
        value2: i32,
    }

    impl Contract {
        #[ink(constructor)]
        pub fn new() -> Self {
            Self {
                value1: 1,
                value2: 2,
            }
        }

        #[ink(message)]
        pub fn get_sum(&self) -> i32 {
            self.value1 + self.value2
        }
    }
}
pub fn new()
Output: [1/*] Building cargo project
 [2/4] Post processing code
 [3/4] Generating metadata
 [4/4] Generating bundle

Original wasm size: 36.6K, Optimized: 11.7K

The contract was built in DEBUG mode.

Your contract artifacts are ready. You can find them in:
c:\Users\asimj\Documents\Polkadot-Seoul\langchain\co

'I have generated an ink contract with the description you provided. The contract has been deployed to Shibuya Testnet and the contract address is `YUc12oBatBfu7rmNzn5Q8WKhfqZh2eoBNiQb45Z59MKpUpa`.'

In [63]:
from dotenv import load_dotenv
import os
import requests
load_dotenv()

base_url = "https://shibuya.api.subscan.io"
substrate_relay = SubstrateInterface(url="wss://shibuya-rpc.dwellir.com")
substrate_contract = SubstrateInterface(
    url='wss://rococo-contracts-rpc.polkadot.io')
api_key = os.getenv("API_KEY")

def get_account_transfers(account_address):
    url = base_url + "/api/scan/transfers"
    headers = {"Content-Type": "application/json", "X-API-Key": api_key}
    payload = {"address": account_address, "row": 100}

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    
    res_list = []

    # Get all the transfers
    transfers = data['data']['transfers']

    # Get the first 3 and last 3 transfers
    selected_transfers = transfers[:3] + transfers[-3:]
    
    for transfer in selected_transfers:

        # Get timestamp and amount
        timestamp = transfer['block_timestamp']
        amount = transfer['amount']

        # Get addresses and truncate
        from_address = transfer['from'][:5] + '...' + transfer['from'][-5:]
        to_address = transfer['to'][:5] + '...' + transfer['to'][-5:]

        res_list.append((f"Timestamp: {timestamp}, Amount: {amount}, From: {from_address}, To: {to_address}"))
    return res_list


In [64]:
get_account_transfers("5CFj55X2XWR1x6DL5W6WUBFmYYB82r37mAw9ukYXsmGcYatd")

['Timestamp: 1690024932, Amount: 0.000001, From: W8KWN...a8WzK, To: YXNdg...8mKXp',
 'Timestamp: 1690024896, Amount: 0.01, From: W8KWN...a8WzK, To: YXNdg...8mKXp',
 'Timestamp: 1690024812, Amount: 0.1, From: W8KWN...a8WzK, To: YXNdg...8mKXp',
 'Timestamp: 1689618036, Amount: 0.000000002, From: W8KWN...a8WzK, To: XZH7C...5BQBn',
 'Timestamp: 1689618036, Amount: 0.000000122, From: W8KWN...a8WzK, To: XZH7C...5BQBn',
 'Timestamp: 1689617520, Amount: 10, From: WiXeZ...wWgQB, To: W8KWN...a8WzK']

In [66]:
agent.run("i want to get all the transactions of 5CFj55X2XWR1x6DL5W6WUBFmYYB82r37mAw9ukYXsmGcYatd")



> Entering new AgentExecutor chain...

Invoking: `list_all_transactions` with `{'account_address': '5CFj55X2XWR1x6DL5W6WUBFmYYB82r37mAw9ukYXsmGcYatd'}`


['Timestamp: 1690024932, Amount: 0.000001, From: W8KWN...a8WzK, To: YXNdg...8mKXp', 'Timestamp: 1690024896, Amount: 0.01, From: W8KWN...a8WzK, To: YXNdg...8mKXp', 'Timestamp: 1690024812, Amount: 0.1, From: W8KWN...a8WzK, To: YXNdg...8mKXp', 'Timestamp: 1689618036, Amount: 0.000000002, From: W8KWN...a8WzK, To: XZH7C...5BQBn', 'Timestamp: 1689618036, Amount: 0.000000122, From: W8KWN...a8WzK, To: XZH7C...5BQBn', 'Timestamp: 1689617520, Amount: 10, From: WiXeZ...wWgQB, To: W8KWN...a8WzK']Here are the last 3 and first 3 transactions of the account with address 5CFj55X2XWR1x6DL5W6WUBFmYYB82r37mAw9ukYXsmGcYatd:

1. Timestamp: 1690024932, Amount: 0.000001, From: W8KWN...a8WzK, To: YXNdg...8mKXp
2. Timestamp: 1690024896, Amount: 0.01, From: W8KWN...a8WzK, To: YXNdg...8mKXp
3. Timestamp: 1690024812, Amount: 0.1, From: W8KWN...a8WzK, To: YXNdg

'Here are the last 3 and first 3 transactions of the account with address 5CFj55X2XWR1x6DL5W6WUBFmYYB82r37mAw9ukYXsmGcYatd:\n\n1. Timestamp: 1690024932, Amount: 0.000001, From: W8KWN...a8WzK, To: YXNdg...8mKXp\n2. Timestamp: 1690024896, Amount: 0.01, From: W8KWN...a8WzK, To: YXNdg...8mKXp\n3. Timestamp: 1690024812, Amount: 0.1, From: W8KWN...a8WzK, To: YXNdg...8mKXp\n\n4. Timestamp: 1689618036, Amount: 0.000000002, From: W8KWN...a8WzK, To: XZH7C...5BQBn\n5. Timestamp: 1689618036, Amount: 0.000000122, From: W8KWN...a8WzK, To: XZH7C...5BQBn\n6. Timestamp: 1689617520, Amount: 10, From: WiXeZ...wWgQB, To: W8KWN...a8WzK'

In [52]:
agent.run("I would like to send 0.001 SBY to 5EenCP39eEQt5uwCxfktmD5d6natP94jK7uuMTnzQ2vBBSC6")



> Entering new AgentExecutor chain...

Invoking: `send_substrate_balance` with `{'recipient_address': '5EenCP39eEQt5uwCxfktmD5d6natP94jK7uuMTnzQ2vBBSC6', 'amount': 0.001}`


Extrinsic "0x8d32669ece6e69c8bdf6b1381b566264ef38e5f0226d67eca1703238b9c88d25" included in block "0x681be6eef200549c99545dd1abb717625eec342f07ff594e2e740c90b4e35713"
✅ Success, triggered events:
* {'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '1f08', 'module_id': 'Balances', 'event_id': 'Withdraw', 'attributes': {'who': 'W8KWNV7Jk48AvFAZoYdVqR3datF14jrnNSJ9TkPk8Ga8WzK', 'amount': 1560011251110461}}, 'event_index': 31, 'module_id': 'Balances', 'event_id': 'Withdraw', 'attributes': {'who': 'W8KWNV7Jk48AvFAZoYdVqR3datF14jrnNSJ9TkPk8Ga8WzK', 'amount': 1560011251110461}, 'topics': []}
* {'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '1f02', 'module_id': 'Balances', 'event_id': 'Transfer', 'attributes': {'from': 'W8KWNV7Jk48AvFAZoYdVqR3datF14jrnNSJ9TkPk8Ga8WzK', 'to'

'I have successfully sent 0.001 SBY to the address 5EenCP39eEQt5uwCxfktmD5d6natP94jK7uuMTnzQ2vBBSC6. The transaction hash is 0x8d32669ece6e69c8bdf6b1381b566264ef38e5f0226d67eca1703238b9c88d25.'

In [80]:
agent.run("Can you get the details of this transaction 0x8d32669ece6e69c8bdf6b1381b566264ef38e5f0226d67eca1703238b9c88d25")



> Entering new AgentExecutor chain...

Invoking: `get_transfer_details` with `{'extrinsic_hash': '0x8d32669ece6e69c8bdf6b1381b566264ef38e5f0226d67eca1703238b9c88d25'}`


['successfully retreived data. Data:', {'code': 0, 'message': 'Success', 'generated_at': 1690027497, 'data': None}]The details of the transaction with the extrinsic hash 0x8d32669ece6e69c8bdf6b1381b566264ef38e5f0226d67eca1703238b9c88d25 are as follows:

- Code: 0
- Message: Success
- Generated At: 1690027497
- Data: null

Please note that the data field is null, indicating that there is no additional data associated with this transaction.

> Finished chain.


'The details of the transaction with the extrinsic hash 0x8d32669ece6e69c8bdf6b1381b566264ef38e5f0226d67eca1703238b9c88d25 are as follows:\n\n- Code: 0\n- Message: Success\n- Generated At: 1690027497\n- Data: null\n\nPlease note that the data field is null, indicating that there is no additional data associated with this transaction.'

In [37]:
res = send_balance("5EenCP39eEQt5uwCxfktmD5d6natP94jK7uuMTnzQ2vBBSC6", 0.001)

Extrinsic "0xf30e17460a83d417ab0b19a965a5d7d7a44d2db30559797c4efd78d5e5c24ed4" included in block "0xfd0e305fca10a791f2d21382b463835d8f6f6cf55c54aa737f38a2f90891d306"
✅ Success, triggered events:
* {'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '1f08', 'module_id': 'Balances', 'event_id': 'Withdraw', 'attributes': {'who': 'W8KWNV7Jk48AvFAZoYdVqR3datF14jrnNSJ9TkPk8Ga8WzK', 'amount': 1560011252740561}}, 'event_index': 31, 'module_id': 'Balances', 'event_id': 'Withdraw', 'attributes': {'who': 'W8KWNV7Jk48AvFAZoYdVqR3datF14jrnNSJ9TkPk8Ga8WzK', 'amount': 1560011252740561}, 'topics': []}
* {'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '1f02', 'module_id': 'Balances', 'event_id': 'Transfer', 'attributes': {'from': 'W8KWNV7Jk48AvFAZoYdVqR3datF14jrnNSJ9TkPk8Ga8WzK', 'to': 'YXNdg1ERU3zJjy3SyD1nsEuBqJ1MMmULKR3bAzrGPv8mKXp', 'amount': 1000000000000}}, 'event_index': 31, 'module_id': 'Balances', 'event_id': 'Transfer', 'attributes': {'from': 'W8KW

In [44]:
res = 

'0xf30e17460a83d417ab0b19a965a5d7d7a44d2db30559797c4efd78d5e5c24ed4'